In [1]:
import pandas as pd
import pickle
from sklearn.metrics import roc_curve
import os
import warnings
import itertools

warnings.filterwarnings("ignore")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import copy
import numpy as np
import seaborn as sns
import torch
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.data import DataLoader, Data
import argparse
import pandas as pd
import json
from sklearn.model_selection import KFold

# from metrics import get_cindex, get_rm2
from train_model import test
from data_utils.dataset import GerNA_dataset2, GerNA_dataset_from_pkl, custom_collate_fn
from net.model import GerNA
from utils.ablation_utils import target_swap, ligand_swap
from ablations import get_indices, identity

In [2]:
# Add argument
datasets = ["Biosensor", "Robin"]
splits = ["random", "RNA", "mol", "both"]


ablations = {
    "target-swap": target_swap,
    "ligand-swap": ligand_swap,
    "none": identity,
}

distributions = {}

data_root = "data"
rows = []
seeds = [0, 1, 2]

for dataset in datasets:

    # model_path = f"save/{mode}/model/epoch-{epochs[mode]}.pt"
    model_path = f"Model/{dataset}_Model_baseline.pth"

    for split_method in splits:

        for ablation_name, ablation in ablations.items():

            rna_dataset_path = f"data/{dataset}/{dataset}_RNA.pkl"
            mol_dataset_path = f"data/{dataset}/{dataset}_Mol.pkl"
            interaction_dataset_path = f"data/{dataset}/{dataset}_interaction.csv"
            my_Dataset = GerNA_dataset_from_pkl(rna_dataset_path, mol_dataset_path, interaction_dataset_path)
            print(f"Initial set:{len(my_Dataset)}")
            test_index = get_indices(dataset, split_method, "test")
            test_set = my_Dataset.smart_subset(test_index)
            print(f"After subsetting:{len(test_set)}")

            for seed in seeds:

                test_set = ablation(test_set, seed=seed)
                print(f"After ablation:{len(test_set)}")
                test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, collate_fn=custom_collate_fn, num_workers=10, pin_memory=True)
                print(f"next(iter(test_loader))={next(iter(test_loader))}")

Initial set:16999
After subsetting:3400
After ablation:3400
next(iter(test_loader))=(tensor([[[-0.0410, -0.1671,  0.1335,  ..., -0.2436, -0.1794, -0.1358],
         [-0.0885, -0.1733,  0.1268,  ..., -0.1306,  0.0369, -0.1140],
         [ 0.1032, -0.1698,  0.0970,  ..., -0.1244,  0.1422,  0.0247],
         ...,
         [ 0.1044, -0.0392,  0.5009,  ..., -0.1095,  0.2101, -0.2479],
         [ 0.1671, -0.1428,  0.3101,  ..., -0.2110,  0.0750, -0.2181],
         [ 0.0914, -0.0150,  0.2405,  ..., -0.0670, -0.0149, -0.2449]],

        [[-0.0410, -0.1671,  0.1335,  ..., -0.2436, -0.1794, -0.1358],
         [-0.0885, -0.1733,  0.1268,  ..., -0.1306,  0.0369, -0.1140],
         [ 0.1032, -0.1698,  0.0970,  ..., -0.1244,  0.1422,  0.0247],
         ...,
         [ 0.1044, -0.0392,  0.5009,  ..., -0.1095,  0.2101, -0.2479],
         [ 0.1671, -0.1428,  0.3101,  ..., -0.2110,  0.0750, -0.2181],
         [ 0.0914, -0.0150,  0.2405,  ..., -0.0670, -0.0149, -0.2449]],

        [[-0.0410, -0.1671,  0.

KeyboardInterrupt: 

In [8]:
len(test_set)

3400